In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
from os import listdir, path
from zipfile import ZipFile
import random
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.image import img_to_array, load_img
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.io import imread
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D, Average,Input
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import math as math

# Any results you write to the current directory are saved as output.

In [ ]:
if not path.exists('../input/data_train/'):
    print('Extracting cat image files...')
    zf = ZipFile('../input/data_train.tar.gz')
    zf.extractall('../input/')
if not path.exists('../input/data_test/'):
    print('Extracting dog image files...')
    zf = ZipFile('../input/data_test.tar.gz')
    zf.extractall('../input/')

# I. Load the data

You first need to create a way to sort the the list dir by alpha-numeral order.

In [ ]:
import csv
import re

def sorted_aphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

Create a function that load the data

In [ ]:
def load_splitted_data(image_size, classes, name_label):
    X_train, Y_train, X_valid = [], [], []
    print("Loading ...")
    with open(path.join('../input/',name_label), 'r') as csvFile:
        reader = list(csv.reader(csvFile))
        reader.pop(0)
        for i, label in reader:
            Y_train.append(label)
    csvFile.close()
    
    class1, class2 = classes[0], classes[1]
    files = sorted_aphanumeric(listdir(path.join('../input/', class1)))
    for i, file in enumerate(files):
        img = load_img(path.join('../input/', class1, file), target_size=image_size)
        X_train.append(img_to_array(img))
        
    files = sorted_aphanumeric(listdir(path.join('../input/', class2)))
    for i, file in enumerate(files):
        img = load_img(path.join('../input/', class2, file), target_size=image_size)
        #print(i, file)
        X_valid.append(img_to_array(img))
    print("Data loaded")
    return np.asarray(X_train, dtype=np.float32), np.asarray(Y_train), np.asarray(X_valid, dtype=np.float32)

In [ ]:
def scale_data(X_tr, X_val, return_scaler=False):
    shape_tr, shape_val = X_tr.shape, X_val.shape
    X_tr_flat = np.ravel(X_tr).reshape(-1, 1)
    X_val_flat = np.ravel(X_val).reshape(-1, 1)
    min_max_scaler = MinMaxScaler()
    X_tr_scaled = min_max_scaler.fit_transform(X_tr_flat).reshape(shape_tr)
    X_val_scaled = min_max_scaler.transform(X_val_flat).reshape(shape_val)
    if not return_scaler:
        return X_tr_scaled, X_val_scaled
    else:
        return X_tr_scaled, X_val_scaled, min_max_scaler
    
def apply_scaling(X, scaler):
    shape_X = X.shape
    X_flat = np.ravel(X).reshape(-1, 1)
    X_scaled = scaler.transform(X_flat).reshape(shape_X)
    return X_scaled

In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    for ax, metric, name in zip(axs, ['acc', 'loss'], ['Accuracy', 'Loss']):
        ax.plot(
            range(1, len(model_history.history[metric]) + 1), 
            model_history.history[metric]
        )
        ax.plot(
            range(1, len(model_history.history['val_' + metric]) + 1), 
            model_history.history['val_' + metric]
        )
        ax.set_title('Model ' + name)
        ax.set_ylabel(name)
        ax.set_xlabel('Epoch')
        ax.legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
from matplotlib import pyplot
from scipy.misc import toimage

def show_imgs(X):
    pyplot.figure(1,figsize=[8,8])
    k = 0
    for i in range(0,4):
        for j in range(0,4):
            pyplot.subplot2grid((4,4),(i,j))
            pyplot.imshow(toimage(X[k]))
            k = k+1
    # show the plot
    pyplot.show()

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003       
    return lrate

def step_decay(epoch):
    initial_lrate = 2e-3
    drop = 0.65
    epochs_drop = 25
    lrate = initial_lrate * math.pow(drop, 
                                    math.floor((1+epoch)/epochs_drop))
    return lrate

def lr_schedule_opti(epoch):
    lrate = 0.001
    drop = 0.65
    epochs_drop = 25
    if epoch > 75:
        lrate = lrate/2
    elif epoch > 100:
        lrate = lrate * math.pow(drop, 
                math.floor((epoch - 50 )/epochs_drop))       
    return lrate

lrate1 = LearningRateScheduler(lr_schedule)
lrate2 = LearningRateScheduler(step_decay)
lrate3 = LearningRateScheduler(lr_schedule_opti)

Load the data. We fix a relatively small image_size (32, 32, 3) to avoid suffering from slow calculation

In [ ]:
#image_size = (32, 32, 3)
image_size = (64, 64, 3)
sample_size = 7200
name_label = "labels_train.csv"

classes = ['data_train/data_train', 'data_test/data_test']
X_train, Y_train, X_test = load_splitted_data(
    image_size=image_size, classes=classes, name_label=name_label)

In [ ]:
print(X_train.shape, X_test.shape, Y_train.shape)
X_train_scaled, X_test_scaled, scaler = scale_data(X_train, X_test, return_scaler=True)
print(X_train_scaled.shape, X_test_scaled.shape)

In [ ]:
show_imgs(X_train[:16])

For Training our model, we will need to validate the data

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

#From label to int
num_classes = 15
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder()
Y_train_LE = label_encoder.fit_transform(Y_train.ravel()).reshape(*Y_train.shape)
#Y_train_LE = Y_train_LE.reshape(-1, 1)
#Y_train_OH = one_hot_encoder.fit_transform(Y_train_LE)

#X_tr, X_val, Y_tr, Y_val = train_test_split(X_train, Y_train_OH, test_size=0.2, random_state=42)
X_tr, X_val, Y_tr, Y_val = train_test_split(X_train, Y_train_LE, test_size=0.2, random_state=42)

In [ ]:
X_tr = X_tr.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
#X_tr = X_tr / 255.0
#X_val = X_val / 255.0
#X_test = X_test / 255.0
#z-score
mean = np.mean(X_tr,axis=(0,1,2,3))
std = np.std(X_tr,axis=(0,1,2,3))
X_tr = (X_tr-mean)/(std+1e-7)
X_val = (X_val-mean)/(std+1e-7)
X_test = (X_test-mean)/(std+1e-7)

# II. Create the neuronal network

In [ ]:
weight_decay = 1e-4
input_shape = X_tr[0,:,:,:].shape
model_input = Input(shape=input_shape)
epochs = 100

def model1(model_input):
    x = (Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))(model_input)
    x = (Activation('elu'))(x)
    x = (BatchNormalization())(x)
    x = (Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (Activation('elu'))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2,2)))(x)
    x = (Dropout(0.3))(x)

    x = (Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (Activation('elu'))(x)
    x = (BatchNormalization())(x)
    x = (Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (Activation('elu'))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2,2)))(x)
    x = (Dropout(0.4))(x)

    x = (Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (Activation('elu'))(x)
    x = (BatchNormalization())(x)
    x = (Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (Activation('elu'))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2,2)))(x)
    x = (Dropout(0.5))(x)

    x = (Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (Activation('elu'))(x)
    x = (BatchNormalization())(x)
    x = (Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (Activation('elu'))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2,2)))(x)
    x = (Dropout(0.7))(x)

    x = (Flatten())(x)
    x = (Dense(num_classes, activation='softmax'))(x)

    model = Model(model_input, x, name='un')
    return model

In [ ]:
def model2(model_input):
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding =    'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(num_classes, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='conv_pool_cnn')
    
    return model

In [ ]:
def model3(model_input):
    x = (Conv2D(32, (3, 3), padding='same'))(model_input)
    #model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2, 2)))(x)
    x = (Dropout(0.25))(x)

    x = (Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=keras.constraints.maxnorm(3)))(x)
    #model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2, 2)))(x)
    x = (Dropout(0.3))(x)

    x = (Conv2D(128, (3, 3), activation='relu', padding='same', kernel_constraint=keras.constraints.maxnorm(3)))(x)
    #model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2, 2)))(x)
    x = (Dropout(0.4))(x)

    x = (Flatten())(x)
    x = (Dense(512, activation='relu', kernel_constraint=keras.constraints.maxnorm(3)))(x)
    x = (Dropout(0.6))(x)
    x = (Dense(15, activation='softmax'))(x)

    model = Model(model_input, x, name='trois')
    return model

In [ ]:
def model4(model_input):
    x = (Conv2D(32, (3, 3), padding='same'))(model_input)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2, 2)))(x)
    x = (Dropout(0.3))(x)

    x = (Conv2D(64, (5, 5), activation='elu', padding='same', kernel_constraint=keras.constraints.maxnorm(3),kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2, 2)))(x)
    x = (Dropout(0.3))(x)

    x = (Conv2D(128, (3, 3), activation='elu', padding='same', kernel_constraint=keras.constraints.maxnorm(3),kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (BatchNormalization())(x)
    x = (MaxPooling2D(pool_size=(2, 2)))(x)
    x = (Dropout(0.4))(x)

    x = (Dense(256, activation='elu', kernel_constraint=keras.constraints.maxnorm(3),kernel_regularizer=regularizers.l2(weight_decay)))(x)
    x = (Dropout(0.6))(x)
    
    x = (Flatten())(x)
    x = (Dense(num_classes, activation='softmax'))(x)

    model = Model(model_input, x, name='quatre')
    return model

In [ ]:
def model5(model_input):
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(num_classes, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
        
    model = Model(model_input, x, name='cinq')
    
    return model

In [ ]:
from sklearn.utils import class_weight

def compile_and_train(model, num_epochs, X_tr, Y_tr, X_val, Y_val): 
    #data augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        )
    datagen.fit(X_tr)

    #training
    batch_size = 128
    num_epoches = num_epochs
    print(X_tr.shape , Y_tr.shape, Y_val.shape)
    dataFlow = datagen.flow(np.array(X_tr), Y_tr, batch_size=batch_size)

    opt = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
    #opt = keras.optimizers.Adam(lr=0.001, decay=1e-5)
    #opt = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    #filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    #checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    #tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    
    model_history = model.fit_generator(dataFlow,
                        steps_per_epoch=X_tr.shape[0] // batch_size,
                        epochs=num_epoches,
                        verbose=1,
                        validation_data=(X_val,Y_val),
                        validation_steps=X_val.shape[0] // batch_size,
                        callbacks=[lrate3])
    
    return model, model_history

In [ ]:
model_1 = model1(model_input)
model_1, _ = compile_and_train(model_1, epochs, X_tr, Y_tr, X_val, Y_val)

In [ ]:
model_2 = model2(model_input)
model_2, _ = compile_and_train(model_2, 70, X_tr, Y_tr, X_val, Y_val)

In [ ]:
model_3 = model3(model_input)
model_3, _ = compile_and_train(model_3, epochs, X_tr, Y_tr, X_val, Y_val)

In [ ]:
model_4 = model4(model_input)
model_4, _ = compile_and_train(model_4, epochs, X_tr, Y_tr, X_val, Y_val)

In [ ]:
model_5 = model5(model_input)
model_5, _ = compile_and_train(model_5, epochs, X_tr, Y_tr, X_val, Y_val)

In [ ]:
def ensemble(models):
    input_img = Input(shape=input_shape)

    outputs = [model(input_img) for model in models] # get the output of model given the input image
    y = Average()(outputs)

    model = Model(inputs=input_img, outputs=y, name='ensemble')
    return model

ensemble_model = ensemble([model_1, model_2, model_3, model_4, model_5])

In [ ]:
def accuracy_ensemble(model):
    Y_pred = model.predict(X_val, batch_size = 32)
    Y_pred = np.argmax(Y_pred, axis=1)
    print(Y_pred[0], Y_val)
    accuracy = 0
    for i in range(Y_val.shape[0]):
        if Y_pred[i] == Y_val[i]:
            accuracy  += 1
    return accuracy/ Y_val.shape[0] 

print(accuracy_ensemble(ensemble_model))

In [ ]:
#testing
print('\nTest result: %.3f' % (accuracy_ensemble(ensemble_model)*100))

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    #for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    #    plt.text(j, i, format(cm[i, j], fmt),
    #             horizontalalignment="center",
    #             color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
Y_val_pred = ensemble_model.predict(X_val, verbose=True)
Y_val_decode = np.argmax(Y_val_pred, axis=1)
Y_val_pred = label_encoder.inverse_transform(Y_val_decode)
Y_val_true = label_encoder.inverse_transform(Y_val)

cnf_matrix = confusion_matrix(Y_val_true, Y_val_pred)
np.set_printoptions(precision=2)

plt.figure(figsize=[10,10])
plot_confusion_matrix(cnf_matrix, classes=list(label_encoder.classes_), normalize=True,
                      title='Normalized confusion matrix')

plt.show()

# III. Predict the Test set and create a csv file

In [ ]:
Y_predict = ensemble_model.predict(X_test, verbose=True)
for i in range(0,100):
    print(Y_predict[i])

In [ ]:
Y_decode = np.argmax(Y_predict, axis=1)
print(list(label_encoder.classes_))
for i in range(0,100):
    print(Y_decode[i])

In [ ]:
Y_predict = label_encoder.inverse_transform(Y_decode)

In [ ]:
data_to_submit = pd.DataFrame({
    'Id':range(0, Y_predict.shape[0]),
    'Category':Y_predict
})
data_to_submit.to_csv('submission_test.csv', index = False)